Import dependencies

In [1]:
import landmark_detector as ld
import os
import random
import numpy as np
import keras
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping

Set variables

In [2]:
words = ['deaf', 'eat', 'fish', 'friend', 'like', 'milk', 'nice', 'no', 'orange', 'teacher', 'want', 'what', 'where', 'yes']
select_words = ['no', 'eat', 'teacher', 'want', 'fish']
path = '../preprocessing/dataset/'
num_features = 126
model_name = 'draft_model'
fps = 20

Get data from mediapipe


In [3]:
def get_data(words, path, detector_path):
    detector = ld.get_detector(detector_path)

    X = []
    y = []

    num_videos = 0
    highest_frame = 0

    bad_videos = 0

    for word in tqdm(words):
        word_path = os.path.join(path, word)
        
        video_files = [f for f in os.listdir(word_path) if f.endswith('.mp4')]
        
        for video_file in tqdm(video_files, desc=word):
            video_path = os.path.join(word_path, video_file)
            
            try:
                video_X = []
                landmarks, current_frames = ld.get_landmarks(video_path, detector)
                
                if len(landmarks) == 0:
                    bad_videos+=1
                    continue
                
                if current_frames > highest_frame:
                    highest_frame = current_frames
                
                for frame in range(len(landmarks)):
                    features = np.array(landmarks[frame]).flatten()
                    video_X.append(features)
                
                X.append(video_X)
                y.append(words.index(word))
                num_videos += 1

            except Exception as e:
                print(f"Error processing video {video_file}: {e}")
                continue 

    return X, y, num_videos, highest_frame, bad_videos

X, y, num_videos, highest_frame, bad_videos = get_data(select_words, path, '../models/hand_landmarker.task')

print('Number of videos:', num_videos)
print('Highest frame:', highest_frame)
print('Videos with no landmarkers detected: ', bad_videos)


  0%|          | 0/5 [00:00<?, ?it/s]

no:   0%|          | 0/53 [00:00<?, ?it/s]

eat:   0%|          | 0/58 [00:00<?, ?it/s]

teacher:   0%|          | 0/58 [00:00<?, ?it/s]

want:   0%|          | 0/49 [00:00<?, ?it/s]

fish:   0%|          | 0/54 [00:00<?, ?it/s]

Number of videos: 227
Highest frame: 180
Videos with no landmarkers detected:  45


Padding and Masking X

In [4]:
def padX(X, num_videos, highest_frame, num_features):
    padded_X = np.zeros((num_videos, highest_frame, num_features))
    mask = np.ones((num_videos, highest_frame, num_features)) 
    for i in range(num_videos):
        video = X[i]
        for j in range(len(video)):
            frame = video[j]
            if len(frame) < num_features:
                padded_X[i, j, :] = np.pad(frame, (0, num_features - len(frame)), 'constant')
                mask[i, j, len(frame):] = 0
            else:
                padded_X[i, j, :] = frame
        if len(video) < highest_frame:
            mask[i, len(video):, :] = 0

    return padded_X, mask

padded_X, mask = padX(X, num_videos, highest_frame, num_features)
print(padded_X.shape)

(227, 180, 126)


Split data

In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(padded_X, y, test_size=0.3, random_state=42) 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) 


X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

Create model

In [6]:
model = keras.Sequential()

model.add(keras.Input(shape=(highest_frame, num_features)))
model.add(layers.Masking(mask_value=0.0))
model.add(layers.LSTM(64))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(select_words), activation='sigmoid'))


model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 180, 126)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,141 (199.77 KB)

 Trainable params: 51,141 (199.77 KB)

 Non-trainable params: 0 (0.00 B)

Train model

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

model.save(f'../models/{model_name}.keras')

with open(f"../models/{model_name}.env", "w") as file:
    file.write(f"MAX_FRAMES={highest_frame}\n")
    file.write(f"NUM_FEATURES={num_features}\n")
    file.write(f"WORDS={",".join(select_words)}\n")
    file.write(f"FPS={fps}\n")

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 140ms/step - accuracy: 0.2161 - loss: 1.6148 - val_accuracy: 0.3235 - val_loss: 1.5374
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.3061 - loss: 1.5404 - val_accuracy: 0.4118 - val_loss: 1.5030
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.2765 - loss: 1.5185 - val_accuracy: 0.3529 - val_loss: 1.4577
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.3348 - loss: 1.4923 - val_accuracy: 0.5294 - val_loss: 1.4125
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.3839 - loss: 1.3846 - val_accuracy: 0.4118 - val_loss: 1.3520
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.3977 - loss: 1.3766 - val_accuracy: 0.5000 - val_loss: 1.2959
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.4582 - loss: 1.3786 - val_accuracy: 0.5294 - val_loss: 1.2820
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5167 - loss: 1.2804 - val_accuracy: 0.5588 - val_loss

Evaluate model

In [8]:
results = model.evaluate(X_test, y_test)

print('Test loss:', results)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6673 - loss: 0.8891
Test loss: [0.9030312299728394, 0.6571428775787354]


Use model

In [9]:
i = random.randint(0,X_test.shape[0]-1)


X_prediction = X_test[i,:,:]
y_prediction = select_words[y_test[i]]

print(model.predict(np.array([X_prediction])))
print("should be", y_prediction)
print("predicted", select_words[np.argmax(model.predict(np.array([X_prediction])))])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
[[0.83057076 0.6637145  0.15750122 0.4398594  0.8741744 ]]
should be fish
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
predicted fish
